In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

In [2]:
from transformers import Qwen3MoeForCausalLM, AutoTokenizer
from tqdm import tqdm
from torch import nn
import torch
import os

/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info


In [3]:
from glob import glob

files = sorted(glob('gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/*'))
files

['gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/319-model_state_dict.pt',
 'gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/639-model_state_dict.pt',
 'gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/959-model_state_dict.pt']

In [4]:
tokenizer = AutoTokenizer.from_pretrained('gfs/01be5b33/Qwen3-30B-A3B-Instruct-2507')

In [5]:
for f in files:
    print(f)

    epoch = os.path.split(f)[1].split('-')[0]

    model = Qwen3MoeForCausalLM.from_pretrained(
        'gfs/01be5b33/Qwen3-30B-A3B-Instruct-2507', 
        torch_dtype="auto",
        device_map="auto",
    )
    
    mapping = torch.load(f, map_location='cpu')
    keys = mapping.keys()
    state_dict = model.state_dict()
    for i in tqdm(range(model.config.num_hidden_layers)):
        A = f'model.layers.{i}.mlp.gate_lora.A'
        if A in mapping:
            B = f'model.layers.{i}.mlp.gate_lora.B'
            a = mapping[A]
            for k in range(a.shape[0]):
                W = f'model.layers.{i}.mlp.experts.{k}.gate_proj.weight'
                W = state_dict[W]
                A_ = mapping[A][k].to(W.device)
                B_ = mapping[B][k].to(W.device)
                m = torch.matmul(A_, B_) * 2.0
                W += m.T.to(W.dtype)
                
        A = f'model.layers.{i}.mlp.up_lora.A'
        if A in mapping:
            B = f'model.layers.{i}.mlp.up_lora.B'
            a = mapping[A]
            for k in range(a.shape[0]):
                W = f'model.layers.{i}.mlp.experts.{k}.up_proj.weight'
                W = state_dict[W]
                A_ = mapping[A][k].to(W.device)
                B_ = mapping[B][k].to(W.device)
                m = torch.matmul(A_, B_) * 2.0
                W += m.T.to(W.dtype)
    
        A = f'model.layers.{i}.mlp.down_lora.A'
        if A in mapping:
            B = f'model.layers.{i}.mlp.down_lora.B'
            a = mapping[A]
            for k in range(a.shape[0]):
                W = f'model.layers.{i}.mlp.experts.{k}.down_proj.weight'
                W = state_dict[W]
                A_ = mapping[A][k].to(W.device)
                B_ = mapping[B][k].to(W.device)
                m = torch.matmul(A_, B_) * 2.0
                W += m.T.to(W.dtype)
    
    keys_lora = [k.split('.lora')[0] for k in keys if '.lora' in k]
    keys_lora = sorted(list(set(keys_lora)))
    for k in tqdm(keys_lora):
        k_ori = k.replace('_orig_mod.', '') + '.weight'
        post_A = '.lora_A'
        post_B = '.lora_B'
        A = k + post_A
        B = k + post_B
        W = state_dict[k_ori]
        A = mapping[A].to(W.device)
        B = mapping[B].to(W.device)
        m = torch.matmul(A.t(), B.t()) * 2.0
        W += m.T.to(W.dtype)
    
    new_path = f'gfs/01be5b33/Qwen3-30B-A3B-Instruct-2507-lora-256-{epoch}'
    model.save_pretrained(new_path)
    tokenizer.save_pretrained(new_path)

    del model, mapping
    torch.cuda.empty_cache()

`torch_dtype` is deprecated! Use `dtype` instead!


gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/319-model_state_dict.pt


100%|███████████████████████████████████████| 192/192 [00:00<00:00, 1807.39it/s]
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/639-model_state_dict.pt


100%|███████████████████████████████████████| 192/192 [00:00<00:00, 2852.64it/s]


gfs-01be5b33-Qwen3-30B-A3B-Instruct-2507-stack/959-model_state_dict.pt


100%|███████████████████████████████████████| 192/192 [00:00<00:00, 2643.40it/s]


In [6]:
# q = """
# Budak itu sangat nakal, pantang orang leka sedikit, duit syiling pun dikebasnya.

# terjemah ke kedah
# """

# system = 'First, you try to think step-by-step in {{lang}}, after that, put your final answer within $\\boxed{}$.'
# messages = [
#     {"role": "system", "content": system.replace('{{lang}}', 'malay')},
#     {"role": "user", "content": q},
# ]

# row = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     tokenize=False
# )
# input_ids = tokenizer(row, add_special_tokens = False, return_tensors = 'pt').to(model.device)['input_ids']
# input_ids

In [7]:
# gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.6, "top_p": None, "top_k": None}

# output_ids = model.generate(input_ids, **gen_kwargs)
# response = tokenizer.batch_decode(output_ids)[0]
# response